In [87]:
import pickle
import networkx as nx
import numpy as np
import math
import EoN
import random

In [88]:
path = '/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung5_Rep1.gem.txt.cell_type.B-cell_ISV2/MST/MST.pkl'
with open(path, 'rb') as handle:
    G = pickle.load(handle)

In [89]:
# G.number_of_nodes()

In [90]:
G.nodes

NodeView(('MZT2A', 'MALAT1', 'DUSP5', 'CD74', 'TYK2', 'HLA-B', 'IGHM', 'HLA-C', 'B2M', 'IGHA1', 'HLA-DRB1', 'HLA-A', 'IGKC', 'HLA-DPA1', 'COL9A2', 'RPL21', 'IGHG1', 'HLA-DRA', 'CXCR4', 'RPL34', 'NPPC', 'NEAT1', 'CD37', 'RPL37', 'IGHG2', 'HLA-DRB5', 'HLA-E', 'CIITA', 'FKBP11', 'MS4A1', 'CD53', 'BTG1', 'HLA-DPB1', 'RPL22', 'TIMP1', 'IL16', 'HLA-DQA1', 'COL1A1', 'HSPA1A', 'HSP90AB1', 'PSAP', 'HSPA1B', 'IGFBP7', 'VIM', 'EZR', 'LTB', 'TNFRSF12A', 'HLA-DQB1', 'IGHD', 'CD79A', 'COL3A1', 'CD52', 'TUBB', 'PTPRC', 'HSP90AA1', 'IFITM3', 'MT2A', 'EIF5A', 'CD19', 'PTGDR2', 'CD3E', 'IRF3', 'KRT14', 'RPL32', 'SRGN', 'PTPRCAP', 'BGN', 'MIF', 'COTL1', 'KLF2', 'CD44', 'GPX1', 'S100A6', 'GSTP1', 'TNFRSF13B', 'IL32', 'BCL2', 'MGP', 'CD83', 'HSPB1', 'JAK1', 'HSP90B1', 'SOD2', 'JCHAIN', 'TAGLN', 'DUSP1', 'CD81', 'NLRC5', 'SELL', 'CCL19', 'CRP', 'CD63', 'VHL', 'CD48', 'RAC2', 'ZFP36', 'H4C3', 'P2RX5', 'JUNB', 'CTNNB1', 'CLU', 'SQSTM1', 'ARHGDIB', 'LGALS1', 'RAMP1', 'LYZ', 'CD27', 'WIF1', 'PTGDS', 'FN1', 'DCN

In [91]:
node_size = nx.pagerank(G, alpha=0.85,weight=None)
sorted_dict = sorted(node_size.items(), key=lambda x: x[1],reverse=True)
labels = {}
for i in sorted_dict[:10]:
    labels[i[0]] = i[0]
labels

{'CD37': 'CD37',
 'CRIP1': 'CRIP1',
 'WIF1': 'WIF1',
 'MMP2': 'MMP2',
 'FABP5': 'FABP5',
 'IFITM3': 'IFITM3',
 'CXCR4': 'CXCR4',
 'FYN': 'FYN',
 'JAK1': 'JAK1',
 'KRT14': 'KRT14'}

In [92]:
root = list(labels.values())[0]

In [93]:
def _hierarchy_pos(G, root, leftmost, width, leafdx = 0.2, vert_gap = 0.2, vert_loc = 0, 
                xcenter = 0.5, rootpos = None, 
                leafpos = None, parent = None):
    # pos: a dict saying where all nodes go if they have been assigned
    # parent: parent of this branch. - only affects it if non-directed

    if rootpos is None:
        rootpos = {root:(xcenter,vert_loc)}
    else:
        rootpos[root] = (xcenter, vert_loc)
    if leafpos is None:
        leafpos = {}
    children = list(G.neighbors(root))
    leaf_count = 0
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)  
    if len(children)!=0:
        rootdx = width/len(children)
        nextx = xcenter - width/2 - rootdx/2
        for child in children:
            nextx += rootdx
            rootpos, leafpos, newleaves = _hierarchy_pos(G,child, leftmost+leaf_count*leafdx, 
                                width=rootdx, leafdx=leafdx,
                                vert_gap = vert_gap, vert_loc = vert_loc-vert_gap, 
                                xcenter=nextx, rootpos=rootpos, leafpos=leafpos, parent = root)
            leaf_count += newleaves

        leftmostchild = min((x for x,y in [leafpos[child] for child in children]))
        rightmostchild = max((x for x,y in [leafpos[child] for child in children]))
        leafpos[root] = ((leftmostchild+rightmostchild)/2, vert_loc)
    else:
        leaf_count = 1
        leafpos[root]  = (leftmost, vert_loc)
#        pos[root] = (leftmost + (leaf_count-1)*dx/2., vert_loc)
#        print(leaf_count)
    return rootpos, leafpos, leaf_count

rootpos, leafpos, leaf_count = _hierarchy_pos(G, root, leftmost=0, width=1)

In [96]:
def my_hierarchy(G, root, parent = None):
    children = list(G.neighbors(root))
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)
    child_list = {}
    if len(children)!=0:
        for child in children:
            # if child == root: print('--------------')
            x = my_hierarchy(G, child, parent=root)
            child_list[child] = x
    return child_list

res = my_hierarchy(G, root)